In [ ]:
import dask.dataframe as dd
import pandas as pd


import random

import pyarrow.parquet as pq


from utils import (
    extract_compounds_by_ids,
    extract_relevant_patents,
    extract_compound_ids_by_patent,
)

In [17]:
PC_MAP_PQ = "../data/SureChEMBL/patent_compound_map.parquet"
COMPOUNDS_PQ = "../data/SureChEMBL/compounds.parquet"
PATENTS_PQ = "../data/SureChEMBL/patents.parquet"

In [ ]:
random.seed(42)
n_random_chunks = 10
random_chunks = [random.randint(1, 200) for _ in range(n_random_chunks)]

In [19]:
patents = extract_relevant_patents(PATENTS_PQ, random_chunks)

chunk164, mask_len=2488, dfs=1
chunk29, mask_len=6243, dfs=2
chunk7, mask_len=7215, dfs=3
chunk190, mask_len=2411, dfs=4
chunk71, mask_len=5062, dfs=5
chunk63, mask_len=9105, dfs=6
chunk58, mask_len=10114, dfs=7
chunk36, mask_len=6399, dfs=8
chunk189, mask_len=2511, dfs=9
chunk27, mask_len=7550, dfs=10


In [20]:
patents_subset = patents.sample(n=40, random_state=42)
patents_subset.to_csv(
    "preprocessing_checkpoints/patents_subset.tsv", sep="\t", index=False
)
patents_id = patents_subset.id.to_list()


In [21]:
compounds_id_df = extract_compound_ids_by_patent(PC_MAP_PQ, patents_id)
compounds_id = compounds_id_df.compound_id.to_list()

compounds_id_df.to_csv(
    "preprocessing_checkpoints/compounds_id_df.tsv", sep="\t", index=False
)

chunk35, mask_len=151, dfs=0
chunk39, mask_len=98, dfs=1
chunk140, mask_len=194, dfs=2
chunk143, mask_len=211, dfs=3
chunk151, mask_len=1173, dfs=4
chunk152, mask_len=209, dfs=5
chunk187, mask_len=719, dfs=6
chunk189, mask_len=295, dfs=7
chunk191, mask_len=775, dfs=8
chunk309, mask_len=1560, dfs=9
chunk311, mask_len=69, dfs=10
chunk313, mask_len=159, dfs=11
chunk334, mask_len=39, dfs=12
chunk335, mask_len=304, dfs=13
chunk336, mask_len=930, dfs=14
chunk337, mask_len=17, dfs=15
chunk338, mask_len=1074, dfs=16
chunk379, mask_len=245, dfs=17
chunk382, mask_len=98, dfs=18
chunk854, mask_len=1, dfs=19
chunk979, mask_len=134, dfs=20
chunk983, mask_len=15, dfs=21
chunk984, mask_len=213, dfs=22


In [22]:
compounds = extract_compounds_by_ids(COMPOUNDS_PQ, compounds_id)
compounds.to_csv("preprocessing_checkpoints/compounds.tsv", sep="\t", index=False)

chunk0, mask_len=1574, dfs=0
chunk1, mask_len=636, dfs=1
chunk2, mask_len=288, dfs=2
chunk3, mask_len=90, dfs=3
chunk4, mask_len=80, dfs=4
chunk5, mask_len=45, dfs=5
chunk6, mask_len=24, dfs=6
chunk7, mask_len=46, dfs=7
chunk8, mask_len=60, dfs=8
chunk9, mask_len=58, dfs=9
chunk10, mask_len=69, dfs=10
chunk11, mask_len=19, dfs=11
chunk12, mask_len=24, dfs=12
chunk13, mask_len=18, dfs=13
chunk14, mask_len=12, dfs=14
chunk15, mask_len=15, dfs=15
chunk16, mask_len=25, dfs=16
chunk17, mask_len=6, dfs=17
chunk18, mask_len=10, dfs=18
chunk19, mask_len=8, dfs=19
chunk20, mask_len=17, dfs=20
chunk21, mask_len=7, dfs=21
chunk22, mask_len=13, dfs=22
chunk23, mask_len=10, dfs=23
chunk24, mask_len=7, dfs=24
chunk25, mask_len=7, dfs=25
chunk26, mask_len=8, dfs=26
chunk27, mask_len=1, dfs=27
chunk28, mask_len=5, dfs=28
chunk29, mask_len=9, dfs=29
chunk30, mask_len=8, dfs=30
chunk31, mask_len=14, dfs=31
chunk32, mask_len=7, dfs=32
chunk33, mask_len=9, dfs=33
chunk34, mask_len=5, dfs=34
chunk35, mask_

In [23]:
comp_with_pat = compounds.merge(
    compounds_id_df,
    left_on="id",
    right_on="compound_id",
    how="left",
)

comp_with_patent_info = comp_with_pat.merge(
    patents,
    left_on="patent_id",
    right_on="id",
    how="left",
    suffixes=("", "_pat"),
)

comp_with_patent_info = comp_with_patent_info.drop(
    ["compound_id", "id_pat"], axis=1
).rename(columns={"id": "compound_id"})


comp_with_patent_info.to_csv(
    "preprocessing_checkpoints/comp_with_patent_info.tsv", sep="\t", index=False
)